In [12]:
import pandas as pd
from functools import reduce
from datetime import datetime

In [13]:
#Load the dataset
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"
df = pd.read_csv(url)

In [19]:
#Select only the needed columns
data = df[['location', 'date', 'people_fully_vaccinated_per_hundred', 'total_vaccinations']]

#Drop rows with missing values in important columns
data = data.dropna(subset=['people_fully_vaccinated_per_hundred', 'total_vaccinations'])

#Use map to convert 'date' from string to datetime objects
#Convert the DataFrame to a list of dicts, then map over each row
mapped_data = list(map(
    lambda row: {**row, 'date': datetime.strptime(row['date'], "%Y-%m-%d")},
    data.to_dict(orient='records')
))

#Get the latest record per country using imperative logic for grouping (not part of map/filter/reduce)
#Group by location and keep the last (latest date) entry
latest_data = {}
for row in mapped_data:
    loc = row['location']
    if loc not in latest_data or row['date'] > latest_data[loc]['date']:
        latest_data[loc] = row

latest_records = list(latest_data.values())




In [18]:
#Use filter to keep only countries where >70% of population is fully vaccinated
filtered_data = list(filter(
    lambda row: row['people_fully_vaccinated_per_hundred'] > 70,
    latest_records
))

# Step 7: Use reduce to sum total vaccinations across the filtered data
total_vaccinations = reduce(
    lambda acc, row: acc + float(row['total_vaccinations']),
    filtered_data,
    0
)

# Final Output
print("Total vaccinations in countries with >70% fully vaccinated population:", int(total_vaccinations))


vaccinated_countries = list(map(lambda row: row['location'], filtered_data))
print("Countries with >70% fully vaccinated population:", vaccinated_countries)

Total vaccinations in countries with >70% fully vaccinated population: 26739296167
Countries with >70% fully vaccinated population: ['Argentina', 'Aruba', 'Asia', 'Australia', 'Austria', 'Bahrain', 'Bangladesh', 'Belgium', 'Bermuda', 'Bhutan', 'Brazil', 'Brunei', 'Cambodia', 'Canada', 'Cayman Islands', 'Chile', 'China', 'Colombia', 'Cook Islands', 'Costa Rica', 'Cuba', 'Cyprus', 'Denmark', 'Ecuador', 'England', 'European Union', 'Faeroe Islands', 'Finland', 'France', 'Germany', 'Gibraltar', 'Greece', 'Guernsey', 'High income', 'Hong Kong', 'Iceland', 'Ireland', 'Isle of Man', 'Italy', 'Japan', 'Jersey', 'Kuwait', 'Latvia', 'Liberia', 'Macao', 'Malaysia', 'Maldives', 'Malta', 'Mauritius', 'Monaco', 'Nauru', 'Nepal', 'New Zealand', 'Nicaragua', 'Niue', 'Northern Cyprus', 'Northern Ireland', 'Norway', 'Panama', 'Peru', 'Pitcairn', 'Portugal', 'Qatar', 'Rwanda', 'Samoa', 'San Marino', 'Scotland', 'Seychelles', 'Singapore', 'South America', 'South Korea', 'Spain', 'Sweden', 'Taiwan', 'Thail

In this vaccination analysis, I used functional programming instead, and I actually found it cleaner and more efficient.

In my code, I used map to convert each country's name into a list. I used a filter to select only the countries where more than 70% of the population was fully vaccinated. Finally, I used reduce to calculate the total number of vaccinations across all of those countries. These functions let me process the data in a more readable and declarative way, without needing to write long loops or worry about manually updating counters.

Functional programming works well in data science because we often deal with big datasets that need transforming, filtering, and summarizing. Tools like map, filter, and reduce make these operations easier to write and understand. Plus, they avoid changing variables, so it’s less likely to cause bugs.

Even though I’m more comfortable with imperative code, I see how functional programming can help write cleaner, shorter, and safer code—especially for tasks like this, where I just want to transform data from one form to another. What I like about functional programming is that it makes data transformations more readable and faster to write. You just tell the computer what you want done instead of how to do it step by step. It’s especially useful when working with big datasets or building data pipelines. Also, since functions like map and filter don’t change the original data, it’s less likely to introduce bugs.